## 1. Import libraries <a class="anchor" id="2"></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
from tqdm import tqdm
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import seaborn as sns
import nltk
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
#from sklearn.metrics.classification import accuracy_score, log_loss
# from sklearn.feature_extraction.text import Tfiall_dataVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
#from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier
from IPython.display import Image

from nltk.corpus import stopwords
from string import punctuation
from gensim import corpora, models, similarities 

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, log_loss
from sklearn.decomposition import PCA
from sklearn import tree
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
%matplotlib inline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

## 2. Download data <a class="anchor" id="3"></a>




In [ ]:
data_variants = pd.read_csv('../input/cancerdata/training_variants')
data_text =pd.read_csv("../input/cancerdata/training_text",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
test_variants = pd.read_csv('../input/cancerdata/test_variants')
test_text =pd.read_csv("../input/cancerdata/test_text",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
cleaned_text_data = pd.read_csv('../input/textcol-new/textcol_new.csv')
cleaned_test_text_data = pd.read_csv('../input/textcol-test/textcol_test.csv')

In [ ]:
all_data = pd.merge(data_variants, data_text,on='ID', how='left')
all_data.head()

In [ ]:
all_test = pd.merge(test_variants, test_text,on='ID', how='left')

In [ ]:
print('There are {} rows and {} columns in train'.format(all_data.shape[0],all_data.shape[1]))
print('There are {} rows and {} columns in test'.format(all_test.shape[0],all_test.shape[1]))

In [ ]:
del test_text
del test_variants
del data_variants
#del data_text

## 3. EDA <a class="anchor" id="4"></a>

In [ ]:
all_data.info()

### Class distribution

In [ ]:
all_data['Class'].value_counts()

In [ ]:
plt.figure(figsize=(16,5))
sns.countplot(y='Class',data=all_data)
plt.gca().xaxis.tick_bottom()
plt.title('Data count by Class')

### Common words in text

In [ ]:
def create_corpus(data):
    corpus=[]
    
    for x in data.dropna(subset=['TEXT'])['TEXT'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

In [ ]:
stop=list(stopwords.words('english'))
stop.extend(['The','the','In','also','fig','Figure','et','al.,','al', 'figure','also', 'et '])

In [ ]:
corpus=create_corpus(all_data)

counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]

for word,count in most[:40]:
    if (word not in stop) :
        x.append(word)
        y.append(count)

In [ ]:
plt.figure(figsize=(16,5))
sns.barplot(x=y,y=x)

### Common stopwords in text

In [ ]:
most=counter.most_common()
x_stop=[]
y_stop=[]

for word,count in most[:20]:
    if (word in stop) :
        x_stop.append(word)
        y_stop.append(count)

In [ ]:
plt.figure(figsize=(16,5))
sns.barplot(x=y_stop,y=x_stop)

### Preprocessing and Cleaning of text

In [ ]:
avg = 0
for i in all_data.index:
    if i in all_data[(all_data['TEXT'].isnull())].index:
        continue
    else:
        avg+=len(all_data['TEXT'][i].split(' '))
print('Average number of words in each text:',avg/3316)

In [ ]:
def initial_clean(text):
    text = text.lower() # lower case the text
    text = nltk.word_tokenize(text)
    return text


def remove_stop_words(text):
    return [word for word in text if word not in stop]

lemmatizer = WordNetLemmatizer()
def stem_words(text):
    try:
        text = [lemmatizer.lemmatize(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    if type(text) is not float:
        return ' '.join(stem_words(remove_stop_words(initial_clean(text))))
    else:
        return text

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
k = all_data['TEXT'].progress_apply(apply_all)
all_data['TEXT'] = k
k.to_csv('./textcol_new.csv')
del k

In [ ]:
cleaned_text_data = pd.read_csv('./textcol_new.csv')

In [ ]:
print("cleaned: \n",cleaned_text_data['TEXT'][1][:257])
print("\n")
print("orignal: \n",all_data['TEXT'][1][:324])

In [ ]:
all_data['TEXT'] = cleaned_text_data['TEXT']
all_test['TEXT'] = cleaned_test_text_data['TEXT']

In [ ]:
del cleaned_text_data
del cleaned_test_text_data

### N-gram analysis

##### We will do a bigram (n=2) analysis over the medical text. Let's check the most common bigrams in text.

In [ ]:
corpus=create_corpus(all_data)

In [ ]:
def get_top_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
top_bigrams=get_top_bigrams(all_data.dropna(subset=['TEXT'])['TEXT'])[:10]

In [ ]:
plt.figure(figsize=(16,5))
x,y=map(list,zip(*top_bigrams))
sns.barplot(x=y,y=x)

##### We will do a trigram (n=3) analysis over the medical text. Let's check the most common trigrams in text.

In [ ]:
def get_top_trigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
top_trigrams=get_top_trigrams(all_data.dropna(subset=['TEXT'])['TEXT'])[:10]

In [ ]:
plt.figure(figsize=(16,5))
x,y=map(list,zip(*top_trigrams))
sns.barplot(x=y,y=x)

### Missing Value imputation

In [ ]:
def LDA(gen, var):
    h = all_data[(all_data['Gene']==gen) | (all_data['Variation']==var)]['TEXT'].values
    corpus = [x for x in h if str(x) != 'nan']
    stoplist = stopwords.words('english') + list(punctuation)
    texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(corpus)
    corpus_tfiall_data = tfidf[corpus] 
    total_topics = 2
    lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
    corpus_lda = lda[corpus_tfiall_data]
    k = lda.show_topics(total_topics,100)
    return k
def final_text(gen,var):
    h = LDA(gen,var)
    reg = re.compile('"(.*?)"')
    ans = reg.findall(h[0][1])
    ans1 = reg.findall(h[1][1])
    ans.extend(ans1)
    fin = ' '.join(ans)
    return fin

for i in tqdm(all_data.loc[all_data['TEXT'].isnull()].index):
    all_data['TEXT'][i] = final_text(all_data['Gene'][i], all_data['Variation'][i])
for i in tqdm(all_test.loc[all_test['TEXT'].isnull()].index):
    all_test['TEXT'][i] = final_text(all_test['Gene'][i], all_test['Variation'][i])

In [ ]:
all_data.isnull().sum()

In [ ]:
del all_test

## 4. WordCloud <a class="anchor" id="6"></a>



In [ ]:
string = data_text['TEXT'].dropna().to_list()
string = ' '.join(string)

In [ ]:
%%time
from wordcloud import WordCloud, STOPWORDS
# Generate word cloud
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='salmon', colormap='Pastel1', 
                      collocations=False, stopwords = STOPWORDS).generate(string)

In [ ]:
def plot_cloud(wordcloud):
    plt.figure(figsize=(40, 30))
    plt.imshow(wordcloud) 
    plt.axis("off");
    
plot_cloud(wordcloud)

In [ ]:
del string

## 5. Splitting dataset into Train, Test and Validation set<a class="anchor" id="6"></a>



In [ ]:
y_true = all_data['Class'].values
X_train, test_df, y_train, y_test = train_test_split(all_data, y_true, stratify = y_true, test_size=0.2)

In [ ]:
train_df, cv_df, y_train, y_cv = train_test_split(X_train,y_train,stratify = y_train, test_size=0.2 )

In [ ]:
print('Data points in train data:', train_df.shape[0])
print('Data points in test data:', test_df.shape[0])
print('Data points in cross validation data:', cv_df.shape[0])

## 6. Word Embedding <a class="anchor" id="8"></a>


### Gene

In [ ]:
vectorizer = TfidfVectorizer()
train_gene_feature_onehotCoding =  vectorizer.fit_transform(train_df['Gene'])
test_gene_feature_onehotCoding  =  vectorizer.transform(test_df['Gene'])
cv_gene_feature_onehotCoding    =  vectorizer.transform(cv_df['Gene'])

### Variant

In [ ]:
train_variation_feature_onehotCoding =  vectorizer.fit_transform(train_df['Variation'])
test_variation_feature_onehotCoding  =  vectorizer.transform(test_df['Variation'])
cv_variation_feature_onehotCoding    =  vectorizer.transform(cv_df['Variation'])

### Text

In [ ]:
text_vectorizer = TfidfVectorizer(min_df=3)
train_text_feature_onehotCoding = text_vectorizer.fit_transform(train_df['TEXT'])
train_text_feature_onehotCoding = normalize(train_text_feature_onehotCoding, axis=0)

test_text_feature_onehotCoding = text_vectorizer.transform(test_df['TEXT'])
test_text_feature_onehotCoding = normalize(test_text_feature_onehotCoding, axis=0)

cv_text_feature_onehotCoding = text_vectorizer.transform(cv_df['TEXT'])
cv_text_feature_onehotCoding = normalize(cv_text_feature_onehotCoding, axis=0)

In [ ]:
del vectorizer

### Stacking all the 3 vectorized forms and making dataframes

In [ ]:
train_gene_var_onehotCoding = hstack((train_gene_feature_onehotCoding,train_variation_feature_onehotCoding))
test_gene_var_onehotCoding = hstack((test_gene_feature_onehotCoding,test_variation_feature_onehotCoding))
cv_gene_var_onehotCoding = hstack((cv_gene_feature_onehotCoding,cv_variation_feature_onehotCoding))

train_x_onehotCoding = hstack((train_gene_var_onehotCoding, train_text_feature_onehotCoding)).tocsr()
train_y = np.array(list(train_df['Class']))

test_x_onehotCoding = hstack((test_gene_var_onehotCoding, test_text_feature_onehotCoding)).tocsr()
test_y = np.array(list(test_df['Class']))

cv_x_onehotCoding = hstack((cv_gene_var_onehotCoding, cv_text_feature_onehotCoding)).tocsr()
cv_y = np.array(list(cv_df['Class']))

## 7. Log Loss evaluation for feature importance <a class="anchor" id="8"></a>


In [ ]:
test_data_len = test_df.shape[0]
cv_data_len = cv_df.shape[0]

# we create a output array that has exactly same size as the CV data
cv_predicted_y = np.zeros((cv_data_len,9))
for i in range(cv_data_len):
    rand_probs = np.random.rand(1,9)
    cv_predicted_y[i] = ((rand_probs/rand_probs.sum())[0])

cv_log_loss = round(log_loss(y_cv,cv_predicted_y, eps=1e-15),2)

print("Log loss on Cross Validation Data using Random Model",cv_log_loss)

test_predicted_y = np.zeros((test_data_len,9))
for i in range(test_data_len):
    rand_probs = np.random.rand(1,9)
    test_predicted_y[i] = ((rand_probs/rand_probs.sum())[0])
test_log_loss = round(log_loss(y_test,test_predicted_y, eps=1e-15),2)

print("Log loss on Test Data using Random Model",test_log_loss)

In [ ]:
def LoglossWithFeat(train_feat_hotencode,cv_feat_hotencode):
    clf = SGDClassifier(alpha=0.1, penalty='l2', loss='log', random_state=42)
    clf.fit(train_feat_hotencode, train_y)
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_feat_hotencode, train_y)
    predict_y = sig_clf.predict_proba(cv_feat_hotencode)
    return log_loss(cv_y, predict_y, labels=clf.classes_, eps=1e-15)

In [ ]:
print("Log loss for Gene feature:",LoglossWithFeat(train_gene_feature_onehotCoding, cv_gene_feature_onehotCoding))
print("Log loss for Text feature:", LoglossWithFeat(train_text_feature_onehotCoding, cv_text_feature_onehotCoding))
print("Log loss for Variant feature:", LoglossWithFeat(train_variation_feature_onehotCoding, cv_variation_feature_onehotCoding))


In [ ]:
del train_gene_feature_onehotCoding,train_variation_feature_onehotCoding
del test_gene_feature_onehotCoding,test_variation_feature_onehotCoding
del cv_gene_feature_onehotCoding,cv_variation_feature_onehotCoding
del train_gene_var_onehotCoding, train_text_feature_onehotCoding
del test_gene_var_onehotCoding, test_text_feature_onehotCoding
del cv_gene_var_onehotCoding, cv_text_feature_onehotCoding

## 8. Modelling <a class="anchor" id="8"></a>


In [ ]:
eval_dict = {}

### Naive Bayes

In [ ]:
%%time
clf = MultinomialNB(alpha=0.1)
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)
sig_clf_probs_train = sig_clf.predict(train_x_onehotCoding)    
sig_clf_probs_test = sig_clf.predict(test_x_onehotCoding)    
sig_clf_probs_cv = sig_clf.predict(cv_x_onehotCoding)    
sig_clf_probs_train_l = sig_clf.predict_proba(train_x_onehotCoding)    
sig_clf_probs_test_l = sig_clf.predict_proba(test_x_onehotCoding)    
sig_clf_probs_cv_l = sig_clf.predict_proba(cv_x_onehotCoding)    

In [ ]:
print(classification_report(train_y,sig_clf_probs_train))
print(classification_report(test_y,sig_clf_probs_test))
print(classification_report(cv_y,sig_clf_probs_cv))
eval_dict['Naive Bayes'] = log_loss(train_y, sig_clf_probs_train_l)

del clf, sig_clf, sig_clf_probs_train, sig_clf_probs_test, sig_clf_probs_cv, sig_clf_probs_train_l, sig_clf_probs_test_l, sig_clf_probs_cv_l

### Random forest

In [ ]:
%%time
clf = RandomForestClassifier(n_estimators=1500, max_depth = 10)
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)
sig_clf_probs_train = sig_clf.predict(train_x_onehotCoding)    
sig_clf_probs_test = sig_clf.predict(test_x_onehotCoding)    
sig_clf_probs_cv = sig_clf.predict(cv_x_onehotCoding)    
sig_clf_probs_train_l = sig_clf.predict_proba(train_x_onehotCoding)    
sig_clf_probs_test_l = sig_clf.predict_proba(test_x_onehotCoding)    
sig_clf_probs_cv_l = sig_clf.predict_proba(cv_x_onehotCoding)    

In [ ]:
print(classification_report(train_y,sig_clf_probs_train))
print(classification_report(test_y,sig_clf_probs_test))
print(classification_report(cv_y,sig_clf_probs_cv))
eval_dict['Random Forest'] = log_loss(train_y, sig_clf_probs_train_l)

del clf, sig_clf, sig_clf_probs_train, sig_clf_probs_test, sig_clf_probs_cv, sig_clf_probs_train_l, sig_clf_probs_test_l, sig_clf_probs_cv_l

### Stochastic Gradient Descent Classifier

In [ ]:
%%time
clf = SGDClassifier(class_weight='balanced', alpha=0.001, penalty='l2', loss='log', random_state=42)
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)
sig_clf_probs_train = sig_clf.predict(train_x_onehotCoding)    
sig_clf_probs_test = sig_clf.predict(test_x_onehotCoding)    
sig_clf_probs_cv = sig_clf.predict(cv_x_onehotCoding)  
sig_clf_probs_train_l = sig_clf.predict_proba(train_x_onehotCoding)    
sig_clf_probs_test_l = sig_clf.predict_proba(test_x_onehotCoding)    
sig_clf_probs_cv_l = sig_clf.predict_proba(cv_x_onehotCoding)    

In [ ]:
print(classification_report(train_y,sig_clf_probs_train))
print(classification_report(test_y,sig_clf_probs_test))
print(classification_report(cv_y,sig_clf_probs_cv))
eval_dict['Logistic Regression with weightbalanced'] = log_loss(train_y, sig_clf_probs_train_l)

del clf, sig_clf, sig_clf_probs_train, sig_clf_probs_test, sig_clf_probs_cv, sig_clf_probs_train_l, sig_clf_probs_test_l, sig_clf_probs_cv_l

### Decision Tree

In [ ]:
%%time
clf = tree.DecisionTreeClassifier()
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)
sig_clf_probs_train = sig_clf.predict(train_x_onehotCoding)    
sig_clf_probs_test = sig_clf.predict(test_x_onehotCoding)    
sig_clf_probs_cv = sig_clf.predict(cv_x_onehotCoding)    
sig_clf_probs_train_l = sig_clf.predict_proba(train_x_onehotCoding)    
sig_clf_probs_test_l = sig_clf.predict_proba(test_x_onehotCoding)    
sig_clf_probs_cv_l = sig_clf.predict_proba(cv_x_onehotCoding)    

In [ ]:
print(classification_report(train_y,sig_clf_probs_train))
print(classification_report(test_y,sig_clf_probs_test))
print(classification_report(cv_y,sig_clf_probs_cv))
eval_dict['Decision Tree'] = log_loss(train_y, sig_clf_probs_train_l)

del clf, sig_clf, sig_clf_probs_train, sig_clf_probs_test, sig_clf_probs_cv, sig_clf_probs_train_l, sig_clf_probs_test_l, sig_clf_probs_cv_l

### LightGBM Classifier

In [ ]:
%%time
clf = LGBMClassifier()
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)
sig_clf_probs_train = sig_clf.predict(train_x_onehotCoding)    
sig_clf_probs_test = sig_clf.predict(test_x_onehotCoding)    
sig_clf_probs_cv = sig_clf.predict(cv_x_onehotCoding)    
sig_clf_probs_train_l = sig_clf.predict_proba(train_x_onehotCoding)    
sig_clf_probs_test_l = sig_clf.predict_proba(test_x_onehotCoding)    
sig_clf_probs_cv_l = sig_clf.predict_proba(cv_x_onehotCoding)    

In [ ]:
print(classification_report(train_y,sig_clf_probs_train))
print(classification_report(test_y,sig_clf_probs_test))
print(classification_report(cv_y,sig_clf_probs_cv))
eval_dict['LGBMClassifier'] = log_loss(train_y, sig_clf_probs_train_l)

del clf, sig_clf, sig_clf_probs_train, sig_clf_probs_test, sig_clf_probs_cv, sig_clf_probs_train_l, sig_clf_probs_test_l, sig_clf_probs_cv_l

## 9. BERT  <a class="anchor" id="10"></a>



In [ ]:
possible_labels = all_data.Class.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
all_data['label']=all_data.Class.replace(label_dict)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(all_data.index.values, 
                                                  all_data.Class.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=all_data.Class.values)

all_data['data_type'] = ['not_set']*all_data.shape[0]

all_data.loc[X_train, 'data_type'] = 'train'
all_data.loc[X_val, 'data_type'] = 'val'

all_data.groupby(['Class', 'data_type']).count()[['TEXT']]

### BertTokenizer and Encoding the Data

In [ ]:
# %%time
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                                          
# encoded_data_train = tokenizer.batch_encode_plus(
#     all_data[all_data.data_type=='train'].TEXT.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt',
#     #truncation=False
# )

In [ ]:
# %%time
# encoded_data_val = tokenizer.batch_encode_plus(
#     all_data[all_data.data_type=='val'].TEXT.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )

In [ ]:
# %%time
# encoded_data_test = tokenizer.batch_encode_plus(
#     all_test.TEXT.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )

In [ ]:
# %%time
# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# labels_train = torch.tensor(all_data[all_data.data_type=='train'].label.values)

# input_ids_val = encoded_data_val['input_ids']
# attention_masks_val = encoded_data_val['attention_mask']
# labels_val = torch.tensor(all_data[all_data.data_type=='val'].label.values)


In [ ]:
# input_ids_test = encoded_data_test['input_ids']
# attention_masks_test = encoded_data_test['attention_mask']

In [ ]:
# dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
# dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
# torch.save(dataset_train, './train')
# torch.save(dataset_val, './val')


In [ ]:
#torch.save(dataset_test, './test')

In [ ]:
dataset_train=torch.load('../input/bert-final/train (1)')
dataset_val=torch.load('../input/bert-final/val (1)')

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup, Adafactor



optimizer = AdamW(model.parameters(),
                  lr=1e-10, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat)



def accuracy_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device('cuda')


model = model.to(device)

In [ ]:
import random

seed_val = 7
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_clf = classification_report_func(predictions, true_vals)
    val_acc = accuracy_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Accuracy: {val_acc}')
    tqdm.write(val_clf)
    

In [ ]:
del model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('../input/models/finetuned_BERT_epoch_5.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)